# Advent of Code 2020 Day 14
[link](https://adventofcode.com/2020/day/14)

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#read-input" data-toc-modified-id="read-input-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>read input</a></span></li><li><span><a href="#part-1" data-toc-modified-id="part-1-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>part 1</a></span><ul class="toc-item"><li><span><a href="#answer" data-toc-modified-id="answer-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>answer</a></span></li></ul></li><li><span><a href="#part-2" data-toc-modified-id="part-2-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>part 2</a></span><ul class="toc-item"><li><span><a href="#answer" data-toc-modified-id="answer-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>answer</a></span></li></ul></li></ul></div>

## read input

In [199]:
function parse_mask(line)
    m = match(r"^mask = ([01X]+)", line)
    return m[1]
end

parse_mask (generic function with 1 method)

In [196]:
function parse_cmd(line)
    m = match(r"^mem\[(\d+)\] = (\d+)", line)
    return (addr = parse(Int, m[1]), val = parse(Int, m[2]))
end

parse_cmd (generic function with 1 method)

In [197]:
function parse_input(filename)
    lines = readlines(filename)
    
    groups = []
    cur_group = nothing
    for l in lines
        if l[1:4] == "mask"
            push!(groups, (mask = parse_mask(l), cmds = []))
            cur_group = last(groups)
        else
            push!(cur_group.cmds, parse_cmd(l))
        end
    end
    
    return groups
end

parse_input (generic function with 1 method)

In [200]:
input_sample_1 = parse_input("input_sample_1.txt")

1-element Array{Any,1}:
 (mask = "XXXXXXXXXXXXXXXXXXXXXXXXXXXXX1XXXX0X", cmds = Any[(addr = 8, val = 11), (addr = 7, val = 101), (addr = 8, val = 0)])

In [201]:
input_sample_2 = parse_input("input_sample_2.txt")

2-element Array{Any,1}:
 (mask = "000000000000000000000000000000X1001X", cmds = Any[(addr = 42, val = 100)])
 (mask = "00000000000000000000000000000000X0XX", cmds = Any[(addr = 26, val = 1)])

In [202]:
input_puzzle = parse_input("input_puzzle.txt")

100-element Array{Any,1}:
 (mask = "01101001XXX000000111X1000X0000X11010", cmds = Any[(addr = 9227, val = 2018), (addr = 2743, val = 107216), (addr = 7335, val = 5498199), (addr = 62141, val = 28643), (addr = 10396, val = 615), (addr = 44596, val = 16631831), (addr = 30301, val = 1036820554)])
 (mask = "0X0010X11100000010011011111010101XX1", cmds = Any[(addr = 25912, val = 1706212), (addr = 18700, val = 1436), (addr = 31216, val = 1307), (addr = 61767, val = 10950397), (addr = 55019, val = 9351804), (addr = 5477, val = 970907882), (addr = 49380, val = 1102790)])
 (mask = "1100110XX10X10X00001X00010X10XX11111", cmds = Any[(addr = 43431, val = 43329), (addr = 39748, val = 2295), (addr = 59160, val = 4068818)])
 (mask = "X10010101XXX00X010010X1X011001001X1X", cmds = Any[(addr = 783, val = 969), (addr = 15182, val = 4439242), (addr = 34082, val = 129773486), (addr = 3726, val = 36911281), (addr = 14093, val = 198), (addr = 7136, val = 967728)])
 (mask = "1100X1100101000011011000010XX010111

## part 1

In [248]:
function set_memory(instruction_groups, mask_func)
    result = Dict{Int, Int}()
    for (mask, cmds) in instruction_groups, (addr, val) in cmds
        for (addr, val) in mask_func(mask, addr, val)
            result[addr] = val
        end
    end
    return result
end

set_memory (generic function with 2 methods)

In [247]:
function apply_value_mask(mask, addr, val)
    val_string = string(val, base=2, pad=36)
    masked_val = map(zip(mask, val_string)) do (m, v)
        return m != 'X' ? m : v
    end
    return [(addr, parse(Int, String(masked_val), base=2))]
end

apply_value_mask (generic function with 2 methods)

### answer

In [231]:
function show_answer_report(input, ::Val{:part1})
    result = set_memory(input, apply_value_mask)
    @info "Answer found." answer=sum(values(result))
    return
end

show_answer_report (generic function with 2 methods)

In [249]:
show_answer_report(input_sample_1, Val(:part1))

┌ Info: Answer found.
│   answer = 165
└ @ Main In[231]:3


In [250]:
show_answer_report(input_puzzle, Val(:part1))

┌ Info: Answer found.
│   answer = 15172047086292
└ @ Main In[231]:3


## part 2

In [264]:
function apply_addr_mask(mask, addr, val)
    addr_string = string(addr, base=2, pad=36)
    masked_addr = map(zip(mask, addr_string)) do (m, a)
        m != '0' ? m : a
    end
    
    addrs = [0]
    for i in 1:length(masked_addr)
        if masked_addr[i] == 'X'
            new_addrs = addrs .+ 2^(length(masked_addr) - i)
            append!(addrs, new_addrs)
        elseif masked_addr[i] == '1'
            addrs .+= 2^(length(masked_addr) - i)
        end
    end
    
    return tuple.(addrs, val)
end

apply_addr_mask (generic function with 1 method)

### answer

In [265]:
function show_answer_report(input, ::Val{:part2})
    mems = set_memory(input, apply_addr_mask)
    @info "Answer found." answer=sum(values(mems))
    return
end

show_answer_report (generic function with 2 methods)

In [258]:
show_answer_report(input_sample_2, Val(:part2))

┌ Info: Answer found.
│   answer = 208
└ @ Main In[251]:3


In [259]:
show_answer_report(input_puzzle, Val(:part2))

┌ Info: Answer found.
│   answer = 4197941339968
└ @ Main In[251]:3


In [262]:
@benchmark sum(values(set_memory(input_puzzle, apply_value_mask)))

BenchmarkTools.Trial: 
  memory estimate:  425.91 KiB
  allocs estimate:  8347
  --------------
  minimum time:     424.192 μs (0.00% GC)
  median time:      485.295 μs (0.00% GC)
  mean time:        555.215 μs (8.17% GC)
  maximum time:     7.201 ms (90.54% GC)
  --------------
  samples:          9001
  evals/sample:     1

In [266]:
@benchmark sum(values(set_memory(input_puzzle, apply_addr_mask)))

BenchmarkTools.Trial: 
  memory estimate:  21.23 MiB
  allocs estimate:  478247
  --------------
  minimum time:     16.262 ms (0.00% GC)
  median time:      22.581 ms (0.00% GC)
  mean time:        22.656 ms (10.09% GC)
  maximum time:     37.516 ms (30.24% GC)
  --------------
  samples:          221
  evals/sample:     1